In [ ]:
# Imports & helper functions
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement

# Helper: generate Fock basis for nsites, nbosons
def gen_basis(nsites, nbosons):
    if nsites == 1:
        return [[nbosons]]
    basis = []
    for k in range(nbosons+1):
        for rest in gen_basis(nsites-1, nbosons-k):
            basis.append([k] + rest)
    return basis

In [ ]:
# Project 1 : 2‑Site Bose–Hubbard ED

In [ ]:
# **Build & Diagonalize**

# Parameters
J = 1.0
U_vals = np.linspace(0, 30, 301)

# 2‑boson on 2‑site basis
basis2 = [(2,0),(1,1),(0,2)]
idx2 = {s:i for i,s in enumerate(basis2)}

def H2(U):
    H = np.zeros((3,3))
    # On‑site U terms
    for (n1,n2),i in idx2.items():
        H[i,i] = 0.5*U*(n1*(n1-1)+n2*(n2-1))
    # Hopping
    for (n1,n2),i in idx2.items():
        if n2>0:
            j = idx2[(n1+1,n2-1)]
            amp = np.sqrt((n1+1)*n2)
            H[i,j] += -J*amp; H[j,i] += -J*amp
    return H

energies = np.array([np.sort(np.linalg.eigvalsh(H2(U))) for U in U_vals])

In [ ]:
# **Plot Spectrum**

plt.figure(figsize=(6,4))
for mode in range(3):
    plt.plot(U_vals, energies[:,mode], label=f"Level {mode}")
plt.xlabel('U/J'); plt.ylabel('Energy')
plt.title('2‑Site BH Spectrum vs U/J')
plt.legend(); plt.grid(True)
plt.savefig('bloch_BH2_spectrum.png', dpi=300)
plt.show()

In [ ]:
# Project 2: 4‑Site Chain Number Fluctuations

In [ ]:
# **Build, Diagonalize & Compute Fluctuations**

# Parameters
J = 1.0
U_vals2 = np.linspace(0, 20, 41)
nsites, nbosons = 4, 4

basis4 = [tuple(s) for s in gen_basis(nsites, nbosons)]
idx4 = {s:i for i,s in enumerate(basis4)}
dim4 = len(basis4)

def H4(U):
    H = np.zeros((dim4, dim4))
    # On‑site U
    for s,i in idx4.items():
        H[i,i] = 0.5*U*sum(n*(n-1) for n in s)
    # Hopping open chain
    for s,i in idx4.items():
        for site in range(nsites-1):
            n, m = s[site], s[site+1]
            if m>0:
                new = list(s); new[site]+=1; new[site+1]-=1
                j = idx4[tuple(new)]
                amp = np.sqrt((n+1)*m)
                H[i,j] += -J*amp; H[j,i] += -J*amp
    return H

fluct = np.zeros((len(U_vals2), nsites))
for k,U in enumerate(U_vals2):
    w,v = np.linalg.eigh(H4(U))
    gs = v[:,0]
    for site in range(nsites):
        # ⟨n²⟩ – ⟨n⟩²
        ps = np.abs(gs)**2
        avg = sum(ps[i]*state[site] for state,i in idx4.items())
        avg2 = sum(ps[i]*state[site]**2 for state,i in idx4.items())
        fluct[k,site] = avg2 - avg**2

In [ ]:
# **Plot Fluctuations**

plt.figure(figsize=(6,4))
for site in range(nsites):
    plt.plot(U_vals2, fluct[:,site], label=f"Site {site+1}")
plt.xlabel('U/J'); plt.ylabel('⟨Δn²⟩')
plt.title('4‑Site Number Fluctuations vs U/J')
plt.legend(); plt.grid(True)
plt.savefig('bloch_BH4_fluctuations.png', dpi=300)
plt.show()